# Computational Exercise

### Objetivo: 

Implementar o Algoritmo 1.2: Plano de Corte Estendido, com qualquer linguagem computacional e usando o Gurobi para resolver os problemas de  Programacao Linear Inteira. 

Testar o algorimo , resolvendo o problema

Min {x^TQx : e'x=20, x \in {0,1}^n}

onde e é o vetor de componentes 1, n=50 e Q é uma matriz semidefinida positiva nxn gerada aleatoriamente. 

Comparar a solução com a solução dada pelo Gurobi para o mesmo problema. 

## Imports

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Set Up

### Creating symmetric positive semidefinite matrix

In [38]:
# Create a new symmetric positive semidefinite matrix
n = 50
Q = np.random.rand(n, n)
Q = np.matmul(Q, Q.transpose())

### Solver

In [122]:
def master_problem(n, Q, L):
    # Creates a new model
    model = gp.Model("milp")

    # Creates variables
    alpha = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="alpha")
    # x = model.addVars(n, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    x = model.addVars(n, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.BINARY, name="x")

    # Defines objective function
    objective = alpha

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraints of objetive function from problem P
    for k in range(len(L)):
        xk = L[k]
        model.addConstr(
            gp.quicksum(Q[i, j] * (x[i] - xk[i]) * xk[j] for i in range(n) for j in range(n))
            + gp.quicksum(Q[i, j] * xk[i] * xk[j] for i in range(n) for j in range(n)) <= alpha,
            name=f"constraint_p_{k+1}"
        )

    # Adds constraints of contraint e^Tx = 20 from problem P
    for k in range(len(L)):
        xk = L[k]
        model.addConstr(
            gp.quicksum(xk[i] * (x[i] - xk[i]) + 1 * xk[i] for i in range(n)) - 20 <= 0,
            name=f"constraint_ep_{k}"
        )
        model.addConstr(
            gp.quicksum(-xk[i] * (x[i] - xk[i]) - 1 * xk[i] for i in range(n)) + 20 <= 0,
            name=f"constraint_en_{k}"
        )

    # Adds contraints of contraint x \in {0,1}^n 
    # for k in range(len(L)):
    #     xk = L[k]
    #     for i in range(0, n):
    #         model.addConstr((1 - 2*xk[i])*(x[i] - xk[i]) + xk[i]*(1 - xk[i]) <= 0, name=f"constraint_bp_{k}_{i+1}")
    #         model.addConstr(-(1 - 2*xk[i])*(x[i] - xk[i]) - xk[i]*(1 - xk[i]) <= 0, name=f"constraint_bn_{k}_{i+1}")

    # Optimizes model
    model.optimize()
    
    # Extracts model variables
    alpha_value = alpha.X 
    x_values = np.array([x[i].X for i in range(n)])

    return [alpha_value, x_values]

In [40]:
def random_viable_point(n):
    if n < 20:
        raise ValueError("Numpy array size has to be at least 20 so that 1^Tx = 20.")
    
    # Creates an array of zeros
    x = np.zeros(n, dtype=int)
    
    # Generates 20 unique indexes
    indexes = np.random.choice(n, size=20, replace=False)
    
    # Define chosen indexes as 1
    x[indexes] = 1
    
    return x

In [54]:
def random_binary_point(n):
    x = np.random.randint(2, size=n)
    return x

In [142]:
def is_viable(x):
    total = 0
    for i in range(len(x)):
        if x not in [0, 1]:
            return False
        else:
            total += x[i]
    if total == 20:
        return True
    return False

In [146]:
def f(x):
    return sum(Q[i, j] * x[i] * x[j] for i in range(n) for j in range(n))

In [ ]:
# L = [random_viable_point(n) for i in range(0, 1)]
L = [np.random.rand(n)]
output = master_problem(n=n, Q=Q, L=L)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Freedesktop SDK 24.08 (Flatpak runtime)")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 51 columns and 151 nonzeros
Model fingerprint: 0x638fadf0
Variable types: 1 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [4e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s
Presolved: 1 rows, 50 columns, 50 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)

Root relaxation: objective 4.702535e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4702.53463    0    1   

In [155]:
def extended_cutting_plane(n, Q, L0, epsilon):
    zl = -np.inf
    zu = np.inf
    x0 = np.random.rand(n)
    L = [elem for elem in L0]
    L.append(x0)
    x_star = None
    while (zu - zl > epsilon):
        alpha, x = master_problem(n=n, Q=Q, L=L)
        zl = alpha
        if is_viable(x) and f(x) < zu:
            x_star = x
            zu = f(x)
        L.append(x)
    return x_star

In [ ]:
L = [np.random.rand(n)]
epsilon = 10 ** -2
x_star = extended_cutting_plane(n=n,Q=Q, L0=L, epsilon=epsilon)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Freedesktop SDK 24.08 (Flatpak runtime)")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 51 columns and 302 nonzeros
Model fingerprint: 0x9929d9b7
Variable types: 1 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [7e-03, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 2 rows, 50 columns, 98 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)

Root relaxation: objective 5.489117e+03, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5489.11675    0    2   